# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [1]:
%pip install -q anywidget ipywidgets "wiki3_ai>=0.5.1"

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [5]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [6]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [7]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [8]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Code flows, clean and bright,
Indentation guides the way,
Logic takes new form. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [9]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

**Explanation:**

1.  **`import csv`**: Imports the `csv` module.
2.  **`with open('your_file.csv', 'r') as file:`**: Opens the CSV file in read mode (`'r'`). The `with` statement ensures the file is properly closed afterward. Replace `'your_file.csv'` with the actual filename.
3.  **`reader = csv.reader(file)`**: Creates a `csv.reader` object to iterate over rows in the CSV file.
4.  **`for row in reader:`**: Iterates through each row in the CSV file.  `row` is a list of strings representing the values in each cell of that row.
5.  **`print(row)`**: Prints each row. You can then process the data in each row as needed.



For more complex CSV files (e.g., with headers or different delimiters), explore `csv.DictReader`.



In [13]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """
  Reads a CSV file and prints each row.

  Args:
    filename: The name of the CSV file to read.
  """
  try:
    # Open the CSV file in read mode ('r'). The 'with' statement ensures the file is automatically closed.
    with open(filename, 'r') as file:
      # Create a CSV reader object. This object allows you to iterate over the rows of the CSV file.
      reader = csv.reader(file)

      # Iterate over each row in the CSV file. Each 'row' is a list of strings, representing the values in each column.
      for row in reader:
        # Print the row to the console. You can replace this with your desired processing logic.
        print(row)
  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
  except Exception as e:
    print(f"An error occurred: {e}")
```


## Streaming Response

Stream the response as it's generated.

In [10]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: 


In [15]:
assistant_session.widget()._stream_chunks

{'db590092-1b1e-4a98-9f32-db11c73a8759': ['Why',
  ' did',
  ' the',
  ' computer',
  ' to',
  '?',
  ' ',
  '\n\n',
  ' had',
  ' too',
  ' issues',
  '!',
  '\n']}

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [11]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: A list comprehension is a concise way to create new lists based on existing iterables (like lists, tuples, or ranges) in Python.  It's a more compact and often faster alternative to using `for` loops to build lists.

Example:  `squares = [x**2 for x in range(10)]` creates a list of squares from 0 to 9.

Assistant: ```python
# Standard loop
numbers = [1, 2, 3, 4, 5]
squares = []
for num in numbers:
  squares.append(num**2)
print(squares)  # Output: [1, 4, 9, 16, 25]

# List comprehension
numbers = [1, 2, 3, 4, 5]
squares = [num**2 for num in numbers]
print(squares)  # Output: [1, 4, 9, 16, 25]
```

The list comprehension `[num**2 for num in numbers]` achieves the same result in a single line.


## Check Token Usage

Monitor how many tokens you've used.

In [12]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 417/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [13]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [14]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

result1 = await branch1.prompt("The dragon was friendly and helpful.")
print("Branch 1:", result1)
print()

Branch 1: Once upon a time, there was a dragon. Not a fearsome, fire-breathing terror of legends, mind you. This dragon, whose name was Zephyr, was rather underwhelming. He didn't breathe scorching flames, but puffs of shimmering, lavender-scented smoke. He wasn't armored in scales of obsidian or ruby, but sported a coat of soft, moss-green scales that often caught dew and sparkled in the sunlight. 

Zephyr lived in a hollowed-out willow tree nestled beside Whisperwind Creek. He spent his days collecting fallen feathers – blue jay feathers, robin feathers, even the iridescent plumage of hummingbirds – and weaving them into elaborate crowns. He didn't hoard gold, but polished pebbles and smooth river stones, arranging them in pleasing patterns around his cozy nest. 

Zephyr was exceptionally friendly and helpful. The creatures of the forest knew they could always rely on him. When the squirrels' winter stores dwindled, Zephyr gently puffed lavender smoke to stimulate the growth of plump

In [15]:
result2 = await branch2.prompt("The dragon was fierce and terrifying.")
print("Branch 2:", result2)

Branch 2: Once upon a time, there was a dragon. But *this* dragon was not like the others. This was Ignis, and terror was not just a part of him – it *was* him. He wasn’t built for shimmering scales and gentle breezes. Ignis was forged from shadow and obsidian, his scales the color of a dying star, etched with veins of molten gold that pulsed with barely contained power.

His roar wasn't a rumble, but a seismic tremor that shattered stone and rattled the very foundations of mountains. Smoke poured from his nostrils, not lavender-scented, but acrid and choking, a harbinger of destruction. His wings, vast and leathery, blotted out the sun as he wheeled through the sky, casting the land below into perpetual twilight.

Ignis didn’t hoard gold; he *demanded* it. Mountains of it piled high in his cavern, shimmering under the eerie glow of volcanic vents. But gold wasn't his obsession. Power was. The power to command, to destroy, to rule. He’d spent centuries honing his fearsome reputation, c

## Cleanup

Destroy sessions when you're done to free up resources.

In [16]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
